In [1]:
import pandas as pd
data = pd.DataFrame([['a',1],['B',None],['C c',3]], columns=['letter','number'])

In [2]:
data

,letter,number
0,a,1.0
1,B,NaN
2,C c,3.0


In [3]:
data['number'].fillna(0)

0    1.0
1    0.0
2    3.0
Name: number, dtype: float64

In [4]:
data['number'].fillna(data['number'].mean())

0    1.0
1    2.0
2    3.0
Name: number, dtype: float64

In [5]:
data['number'].fillna(data['letter'])

0    1
1    B
2    3
Name: number, dtype: object

In [6]:
data = pd.DataFrame([['a',1],['B',None],['Cat c',3]], columns=['letter','number'])

In [7]:
data

,letter,number
0,a,1.0
1,B,NaN
2,Cat c,3.0


In [8]:
data['letter'].str.lower()

0        a
1        b
2    cat c
Name: letter, dtype: object

In [9]:
data['letter'].str.upper()

0        A
1        B
2    CAT C
Name: letter, dtype: object

In [10]:
data['letter'].str.title()

0        A
1        B
2    Cat C
Name: letter, dtype: object

In [11]:
data['letter'].str.replace('^.* ','')

0    a
1    B
2    c
Name: letter, dtype: object

# Situation / Problem

In this data set, we want to group by the Proprietary Name.  How many drugs are represented here?

* Does every row have a proprietary name? - If not, use the non-proprietary name instead
* Are they formatted consistently? - Make them all upper case or lower case
* Are they sometimes more specific than we want? - Look for extra stuff like units of measure

In [12]:
import pandas as pd

In [13]:
ndc = pd.read_csv('/data/ndc.txt', delimiter='\t')

In [14]:
ndc

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,N,20191231.0
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,HUMAN PRESCRIPTION DRUG,NaN,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0
2,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,HUMAN PRESCRIPTION DRUG,Quinidine Gluconate,NaN,Quinidine Gluconate,SOLUTION,INTRAVENOUS,19500712,NaN,NDA,NDA007529,Eli Lilly and Company,QUINIDINE GLUCONATE,80,mg/mL,"Antiarrhythmic [EPC],Cytochrome P450 2D6 Inhib...",NaN,N,20191231.0
3,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,0002-1434_4468578a-47d2-488e-9fd4-a8322070392f,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0003-0852_9bc9427b-2cca-466c-b41e-d47ce4540aa6,0003-0852,HUMAN PRESCRIPTION DRUG,SPRYCEL,NaN,dasatinib,TABLET,ORAL,20080530,NaN,NDA,NDA021986,"E.R. Squibb & Sons, L.L.C.",DASATINIB,100,mg/1,"Kinase Inhibitor [EPC],Protein Kinase Inhibito...",NaN,N,20201231.0
90,0003-0855_9bc9427b-2cca-466c-b41e-d47ce4540aa6,0003-0855,HUMAN PRESCRIPTION DRUG,SPRYCEL,NaN,dasatinib,TABLET,ORAL,20101028,NaN,NDA,NDA021986,"E.R. Squibb & Sons, L.L.C.",DASATINIB,80,mg/1,"Kinase Inhibitor [EPC],Protein Kinase Inhibito...",NaN,N,20201231.0
91,0003-0857_9bc9427b-2cca-466c-b41e-d47ce4540aa6,0003-0857,HUMAN PRESCRIPTION DRUG,SPRYCEL,NaN,dasatinib,TABLET,ORAL,20101028,NaN,NDA,NDA021986,"E.R. Squibb & Sons, L.L.C.",DASATINIB,140,mg/1,"Kinase Inhibitor [EPC],Protein Kinase Inhibito...",NaN,N,20201231.0
92,0003-0893_391cf576-1ae6-4a11-bf14-8e5a135cd5dd,0003-0893,HUMAN PRESCRIPTION DRUG,eliquis,NaN,apixaban,"TABLET, FILM COATED",ORAL,20121228,NaN,NDA,NDA202155,"E.R. Squibb & Sons, L.L.C.",APIXABAN,2.5,mg/1,"Factor Xa Inhibitor [EPC],Factor Xa Inhibitors...",NaN,N,20201231.0


In [15]:
# STEP 1: Are any of them blank

blank_filter = ndc['PROPRIETARYNAME'].isnull()
blanks = ndc[blank_filter]
blanks.shape

(7, 20)

In [16]:
blanks

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,HUMAN PRESCRIPTION DRUG,NaN,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0
5,0002-1436_ad6f74e8-b0ef-4a96-9249-c1225c5cd6a7,0002-1436,HUMAN PRESCRIPTION DRUG,NaN,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,Eli Lilly and Company,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0
10,0002-3228_f1ee27ad-c38e-430a-b027-0b7ebad7403e,0002-3228,HUMAN PRESCRIPTION DRUG,NaN,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20021126,NaN,NDA,NDA021411,Eli Lilly and Company,ATOMOXETINE HYDROCHLORIDE,25,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20201231.0
11,0002-3229_f1ee27ad-c38e-430a-b027-0b7ebad7403e,0002-3229,HUMAN PRESCRIPTION DRUG,NaN,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20021126,NaN,NDA,NDA021411,Eli Lilly and Company,ATOMOXETINE HYDROCHLORIDE,40,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20201231.0
13,0002-3231_89af030d-8f23-4e54-92f0-724df4d81712,0002-3231,HUMAN PRESCRIPTION DRUG,NaN,NaN,Olanzapine and Fluoxetine hydrochloride,CAPSULE,ORAL,20031224,NaN,NDA,NDA021520,Eli Lilly and Company,OLANZAPINE; FLUOXETINE HYDROCHLORIDE,6; 25,mg/1; mg/1,"Atypical Antipsychotic [EPC],Serotonin Reuptak...",NaN,N,20201231.0
14,0002-3232_89af030d-8f23-4e54-92f0-724df4d81712,0002-3232,HUMAN PRESCRIPTION DRUG,NaN,NaN,Olanzapine and Fluoxetine hydrochloride,CAPSULE,ORAL,20031224,20190731.0,NDA,NDA021520,Eli Lilly and Company,OLANZAPINE; FLUOXETINE HYDROCHLORIDE,12; 25,mg/1; mg/1,"Atypical Antipsychotic [EPC],Serotonin Reuptak...",NaN,N,NaN
28,0002-4182_1606b529-d77d-41d8-9379-094caf0241c2,0002-4182,HUMAN PRESCRIPTION DRUG,NaN,NaN,baricitinib,"TABLET, FILM COATED",ORAL,20180531,NaN,NDA,NDA207924,Eli Lilly and Company,BARICITINIB,2,mg/1,"Janus Kinase Inhibitor [EPC],Janus Kinase Inhi...",NaN,N,20191231.0


In [17]:
ndc['PROPRIETARYNAME'] = ndc['PROPRIETARYNAME'].fillna(ndc['NONPROPRIETARYNAME'])

In [18]:
blanks = ndc[ndc['PROPRIETARYNAME'].isnull()]
blanks.shape

(0, 20)

In [19]:
ndc[ndc['PROPRIETARYNAME'] == 'Florbetapir F 18']

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,HUMAN PRESCRIPTION DRUG,Florbetapir F 18,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0


In [20]:
blanks

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH


In [21]:
# STEP 2: Check the formats - how many are all caps, how many are all lowercase

all_caps = ndc[ndc['PROPRIETARYNAME'].str.contains('^[A-Z0-9 \-]+$')]
all_caps.shape

(35, 20)

In [22]:
all_lc = ndc[ndc['PROPRIETARYNAME'].str.contains('^[a-z0-9 \-]+$')]
all_lc.shape

(8, 20)

In [23]:
is_lower = ndc['PROPRIETARYNAME'] == ndc['PROPRIETARYNAME'].str.lower()
ndc[is_lower].shape

(8, 20)

In [24]:
# Let's just make them all lower case

ndc['PROPRIETARYNAME'] = ndc['PROPRIETARYNAME'].str.lower()

In [25]:
all_lc = ndc[ndc['PROPRIETARYNAME'] == ndc['PROPRIETARYNAME'].str.lower() ]
all_lc.shape

(94, 20)

In [26]:
ndc.shape

(94, 20)

In [27]:
# STEP 3: Look for any that might have strange characters or number

weird = ndc[ndc['PROPRIETARYNAME'].str.contains('[0-9\-\\\/]')]
weird.shape

(3, 20)

In [28]:
weird

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,HUMAN PRESCRIPTION DRUG,florbetapir f 18,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0
82,0003-0293_0348e127-4e8a-4512-9489-876955f35e6c,0003-0293,HUMAN PRESCRIPTION DRUG,kenalog-40,NaN,TRIAMCINOLONE ACETONIDE,"INJECTION, SUSPENSION",INTRA-ARTICULAR; INTRAMUSCULAR,20090601,NaN,NDA,NDA014901,"E.R. Squibb & Sons, L.L.C.",TRIAMCINOLONE ACETONIDE,40,mg/mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20191231.0
84,0003-0494_b9871f95-7f8c-4bad-b21a-c2595cab0dc3,0003-0494,HUMAN PRESCRIPTION DRUG,kenalog-10,NaN,TRIAMCINOLONE ACETONIDE,"INJECTION, SUSPENSION",INTRA-ARTICULAR; INTRALESIONAL,20090601,NaN,NDA,NDA012041,"E.R. Squibb & Sons, L.L.C.",TRIAMCINOLONE ACETONIDE,10,mg/mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20191231.0


In [ ]:
# Let's assume kenalog-40 and kenalog-10 should be grouped together

In [29]:
ndc['PROPRIETARYNAME'] = ndc['PROPRIETARYNAME'].str.replace('\-[0-9]+','')

In [30]:
ndc[ndc['PROPRIETARYNAME'].str.contains('kenalog')]

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
82,0003-0293_0348e127-4e8a-4512-9489-876955f35e6c,0003-0293,HUMAN PRESCRIPTION DRUG,kenalog,NaN,TRIAMCINOLONE ACETONIDE,"INJECTION, SUSPENSION",INTRA-ARTICULAR; INTRAMUSCULAR,20090601,NaN,NDA,NDA014901,"E.R. Squibb & Sons, L.L.C.",TRIAMCINOLONE ACETONIDE,40,mg/mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20191231.0
84,0003-0494_b9871f95-7f8c-4bad-b21a-c2595cab0dc3,0003-0494,HUMAN PRESCRIPTION DRUG,kenalog,NaN,TRIAMCINOLONE ACETONIDE,"INJECTION, SUSPENSION",INTRA-ARTICULAR; INTRALESIONAL,20090601,NaN,NDA,NDA012041,"E.R. Squibb & Sons, L.L.C.",TRIAMCINOLONE ACETONIDE,10,mg/mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20191231.0
